# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Saying a little, doing a lot”

> …in F#, the overall emphasis is to enable you to solve _complex_ problems with _simple_ code. You want to focus on solving the problem at hand without neccessarily having to _first_ think about design patterns within which you can put your code, or complex syntax.


## Binding values in F#&nbsp;

> The `let` keyword is the single most important keyword in the F# language.

`let` makes _composition_ in F# possible.


In [ ]:
let age = 35 // binding a value

let website = System.Uri "http://fsharp.org" // binding a reference

let add (first, second) = first + second // binding a refernce to a function

;;

> Here are some takeaways from that small sample:
>
> - _No types_…
> - _No `new` keyword_—In F#, the `new` keyword is optional and generally not used when constructing objects that implement `IDisposable`. Instead, F# views a constructor as a function…
> - _No semicolons_…
> - _No brackets for function arguments_… F# has two ways to define function arguments, known as _tupled form_ and _curried form_. …functions that take a _single_ argument don’t _need_ round brackets (a.k.a. parentheses)…


## `let` isn’t `var`!

> …`let` binds an immutable value to a symbol. The closest thing in C# would be to declare every variable with the `readonly` keyword… It’s best to think of `let` bindings as copy-and-paste [directives](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/preprocessor-directives#defining-symbols); wherever you see the symbol, replace it with the value that was originally assigned during the declaration.
>
> …you can execute the same `let` binding multiple times… within the same scope:


In [ ]:
let foo() =
    let x = 10
    printfn "%d" (x + 20)
    let x = "test" // the previous binding is now out of scope
    let x = 50.0
    x + 200.0

foo () // functions with zero arguments need round brackets

30

250

> This is known as _shadowing_… a more advanced (and somewhat controversial) feature…

This _shadowing_ is why Issac takes care to warn us that comparing `let` to `readonly` in C# is not really leading to better understanding (read the book for his warning!). This _shadowing_ is why a C# programmer would assume `let` _is_ `var`.

>This is useful mainly whan you have some calculation that calculates some new value that should be [used] by all subsequent calculations. Shadowing allows you to hide the previous value, so there is no danger you will accidentaly use the original one.
>
> — [Tomas Petricek](https://stackoverflow.com/a/2478445)

The sounds to me like shadowing is the referencing and immediate _orphaning_ of anonymous immutable values. The feels like a foot-gun for memory leaks in some sophisticated recusrion-based design.


## Scoping values

Similar to Python, F# marks a scope with consistent, columnar whitespace (indentation).

My preference is four spaces:


In [ ]:
open System

let doStuffWithTwoNumbers(first, second) =
    let added = first + second
    Console.WriteLine("{0} + {1} = {2}", first, second, added)
    let doubled = added * 2
    doubled

doStuffWithTwoNumbers (2, 3)

2 + 3 = 5


10

Note that, since 2018 (when Isaac wrote this book), F# supports the same advanced string interpolation stuff we see in ‘modern’ C#:


In [ ]:
let doStuffWithTwoNumbers(first, second) =
    let added = first + second
    Console.WriteLine $"{first} + {second} = {added}" // ‘modern’ interpolation
    let doubled = added * 2
    doubled

doStuffWithTwoNumbers (2, 3)

2 + 3 = 5


10

>You’ll also notice a few more things from this multiline function:
>
>- _No return keyword_… F# assumes that the final expression of a scope is the result of that scope.
>- _No accessibility modifier_—In F#, `public` is the default for top-level values… with nested scopes… you can hide values [below the top level] effectively…
>- _No static modifier_… static is the default way of working in F#. [C# programmers who write lots of [extension methods](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/extension-methods) immediately understand this default.]


## Nested functions

>…F# treats _functions_ as _values_. This means that you can also create functions within other functions! Here’s an example of how to do this in F#:


In [ ]:
let estimateAges(familyName, year1, year2, year3) =
    let calculateAge yearOfBirth =
        let year = System.DateTime.Now.Year
        year - yearOfBirth

    let estimatedAge1 = calculateAge year1
    let estimatedAge2 = calculateAge year2
    let estimatedAge3 = calculateAge year3

    let averageAge = (estimatedAge1 + estimatedAge2 + estimatedAge3) / 3

    sprintf $"Average age for family {familyName} is {averageAge}"

estimateAges ("Wilhite", 1990, 2002, 2004)

Average age for family Wilhite is 22

>The ability to create  nested functions means you can start to think of functions and classes that have a single public method as _interchangeable_…:

| Class | Function |
|- |-
| Constructor / single public method | Arguments passed to the function |
| Private fields | Local values |
| Private methods | Local functions |


## Cyclical dependencies in F#&nbsp;

In F#, this throws an error:


In [ ]:
let f x = g + 1
let g y = y + 2

g 2

Error: input.fsx (1,11)-(1,12) typecheck error The value or constructor 'g' is not defined.

> In F#, _the order in which types are defined matters_. Type A can’t reference Type B if Type A is declared before Type B, and the same applies to values.


@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.235701+3881a96164de75fca84f5f11027f3606b7878044Build date: 2021-07-11T04:06:39.6100964Zhttps://github.com/dotnet/interactive
